# Generate a Summmary spreadsheet file listing all the rows with the Optimize3 protocol
## find files
## write the results to an excel file

In [1]:
from pathlib import Path
import numpy as np
import tables
import pandas as pd
import ccModules3 as cc3

In [2]:
parameters_file = 'Project_info.ods'

In [3]:
#get target folder path
target_folder = Path.cwd().parent
print(target_folder)


/media/bushey/202010/JData/C/C07/C07_Data1


In [4]:
parameters = cc3.read_spread( str(target_folder / parameters_file), index_col=0, header=0)

In [5]:
#parameters needed
output = str(target_folder / parameters['File'].loc['Compiled Experiments'])
cross_file = str(target_folder /parameters['File'].loc['Crosses'])
ods_file_selection = parameters['File'].loc['Excel file tags']
trial_selection = parameters['File'].loc['Trial Selection']

In [6]:
#get list of files in targetfolder
target_files = [i for i in target_folder.glob('**/*' + ods_file_selection)]
target_files

[PosixPath('/media/bushey/202010/JData/C/C07/C07_Data1/20220302/20220302_C07.xlsx'),
 PosixPath('/media/bushey/202010/JData/C/C07/C07_Data1/20220303/20220303_C07.xlsx'),
 PosixPath('/media/bushey/202010/JData/C/C07/C07_Data1/20220308/20220308_C07.xlsx')]

In [7]:
frames = []
for cfile in target_files:
    if 'ods' in str(cfile):
        csheet = pd.read_excel(str(cfile), 0, engine='odf')
    else:
        csheet = pd.read_excel(str(cfile), 0, engine='openpyxl')
    frames.append(csheet)
frames = pd.concat(frames)
frames.dropna(axis = 0, subset = ['Imaging Protocol'], inplace = True)
frames = frames[frames['Imaging Protocol'].str.contains(trial_selection)]
frames.reset_index(inplace=True)
frames.drop(columns = ['index'], inplace=True)

In [8]:
#add in cross and genotype designations for counts
cross_frame = cc3.read_spread(cross_file, header=0)
cross_frame

genotypes = {}
for row, dseries in cross_frame.iterrows():
    cross = f'{dseries["Project"]}-{dseries["Cross#"]}' 
    genotypes[cross] = dseries['Genotype']

response_direction = {}
for ckey in genotypes:
    response_direction[ckey] = 'pos'

In [9]:
#add genotypes to genotype columns
genotype = {}
cross = {}
for row, dseries in frames.iterrows():
    for ccross in genotypes:
        if ccross in dseries['Sample Name']:
            genotype[row] = genotypes[ccross]
            cross[row] = ccross
frames['Genotype'] = pd.Series(genotype)
frames['Cross'] = pd.Series(cross)

In [10]:
counts = frames.groupby(['Genotype', 'Cross']).count()
counts['Sample Name'].to_csv(str( target_folder / 'Counts.csv'))

In [11]:
counts['Sample Name']

Genotype                      Cross
SS87269                       C07-3    5
w;Gr64f-Gal4; Gr64f-Gal4/TM3  C07-4    2
Name: Sample Name, dtype: int64

In [12]:
cc3.write_spread(frames, output)

In [13]:
print(output)

/media/bushey/202010/JData/C/C07/C07_Data1/C07_Data1_Summary.ods


In [14]:
frames

,No.,Sample Name,Genotype,Date,Note1,Stage,Stage Start,Media,Starvation start,Rearing Temp,...,Notes2,Lamina Present,Saline,Baseline Shift,Sample Movement,Responds,Random,Grade,Notes2.1,Cross
0,1,flya_20220302_C07-4,w;Gr64f-Gal4; Gr64f-Gal4/TM3,20220302.0,see responses during LED and odor,Adult,2022-02-23,1:500 Retinal Cornmeal,2022-02-28,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-4
1,NaN,flyb_20220302_C07-4,w;Gr64f-Gal4; Gr64f-Gal4/TM3,20220302.0,LED was not plugged in no stim light,NaN,2022-02-23,1:500 Retinal Cornmeal,2022-02-28,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,did not draw rois,C07-4
2,1,flya_20220303_C07-3,SS87269,20220303.0,Had to restart at first MCH exposure because s...,Adult,2022-02-24,1:500 Retinal Cornmeal,2022-03-02,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-3
3,NaN,flyb_20220303_C07-3,SS87269,20220303.0,NaN,Adult,2022-02-24,1:500 Retinal Cornmeal,2022-03-02,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3
4,1,flya_20220308_C07-3,SS87269,20220308.0,NaN,Adult,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-3
5,NaN,flyb_20220308_C07-3,SS87269,20220308.0,can see alpha1 but gamma4 beta2 obscured,Adult,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3
6,NaN,flyc_20220308_C07-3,SS87269,20220308.0,NaN,NaN,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3


In [15]:
frames

,No.,Sample Name,Genotype,Date,Note1,Stage,Stage Start,Media,Starvation start,Rearing Temp,...,Notes2,Lamina Present,Saline,Baseline Shift,Sample Movement,Responds,Random,Grade,Notes2.1,Cross
0,1,flya_20220302_C07-4,w;Gr64f-Gal4; Gr64f-Gal4/TM3,20220302.0,see responses during LED and odor,Adult,2022-02-23,1:500 Retinal Cornmeal,2022-02-28,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-4
1,NaN,flyb_20220302_C07-4,w;Gr64f-Gal4; Gr64f-Gal4/TM3,20220302.0,LED was not plugged in no stim light,NaN,2022-02-23,1:500 Retinal Cornmeal,2022-02-28,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,did not draw rois,C07-4
2,1,flya_20220303_C07-3,SS87269,20220303.0,Had to restart at first MCH exposure because s...,Adult,2022-02-24,1:500 Retinal Cornmeal,2022-03-02,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-3
3,NaN,flyb_20220303_C07-3,SS87269,20220303.0,NaN,Adult,2022-02-24,1:500 Retinal Cornmeal,2022-03-02,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3
4,1,flya_20220308_C07-3,SS87269,20220308.0,NaN,Adult,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,C07-3
5,NaN,flyb_20220308_C07-3,SS87269,20220308.0,can see alpha1 but gamma4 beta2 obscured,Adult,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3
6,NaN,flyc_20220308_C07-3,SS87269,20220308.0,NaN,NaN,2022-03-02,1:500 Retinal Cornmeal,2022-03-07,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C07-3
